In [5]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
from definitions import *
from tqdm import tqdm
import cv2
from triangles import TRIANGLES
import random

In [5]:
mean_neutral_face_path = PROCESSED_PATH + "/00/mean-lms-00.csv"
neutral_mask = pd.read_csv(mean_neutral_face_path, index_col=0)

In [6]:
df_pav = pd.read_csv("./points_above_average.csv",index_col=0)

In [7]:
filtered_neutral_mask = neutral_mask[neutral_mask.index.isin(df_pav["points_above_average"].tolist())]

In [8]:
points = filtered_neutral_mask.values

In [7]:
kmeans_gruoups_qtd = 6

In [21]:
kmeans = KMeans(n_clusters=kmeans_gruoups_qtd, random_state=0).fit(points)

In [22]:
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 4, 1, 1, 1, 4, 1, 1, 4, 2, 2, 2,
       2, 4, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 4, 4, 4, 5, 5, 5, 5, 5, 5, 3,
       0, 3, 3, 5, 5, 5, 3, 5, 5, 5, 5, 3, 0, 0, 0, 0, 3, 3, 0, 5, 5, 5,
       5, 5, 5, 5, 5, 0, 0, 0, 3, 3, 3])

In [23]:
df_pav["kmean_group"] = kmeans.labels_

In [24]:
df_pav.to_csv("points_above_average.csv")

In [6]:
df_pav = pd.read_csv("points_above_average.csv")

In [8]:
pav_groups = []

pav_list = df_pav["points_above_average"].tolist()
group_list = df_pav["kmean_group"].tolist()
for i in range(kmeans_gruoups_qtd):
    gp = []
    for j in range(len(pav_list)):
        if(group_list[j] == i):
            gp.append(pav_list[j])
    pav_groups.append(gp)


In [9]:
colors = []
for i in range(kmeans_gruoups_qtd):
    colors.append((random.randint(0, 255),random.randint(0, 255),random.randint(0, 255)))

In [27]:
image = np.zeros(
            (int(final_image_size_height), int(final_image_size_width), 3), dtype='uint8')
x = (neutral_mask["x"]).tolist()
y = (neutral_mask["y"]).tolist()
lms = []
for i in range(len(x)):
    lms.append([int(x[i]), int(y[i])])
for i,lm in enumerate(lms):
    color = (255,255,255)
    if(i in pav_list):
        for j in range(len(pav_groups)):
            for k in pav_groups[j]:
                if(i == k):
                    color = colors[j]
                    image = cv2.putText(image, str(i), (lm), cv2.FONT_HERSHEY_SIMPLEX,0.4,color,1,cv2.LINE_AA)
    else:
        image = cv2.circle(image, (lm), radius=5, color=color, thickness=-1)


for i,color in enumerate(colors):
    image = cv2.putText(image, 'OpenCV', (50,50*i + 50), cv2.FONT_HERSHEY_SIMPLEX, 
                   1, color, 2, cv2.LINE_AA)
cv2.imwrite("./kmeans_gps_visualization.png", image)

True